# Teachablemachine
In diesem Notebook trainieren wir ein Modell mit Hilfe von Webcam Bilder. Trainiere hierfür ein Modell auf **https://teachablemachine.withgoogle.com/train/image** und lade es anschließend als "keras"-Modell herunter (Model exportieren --> Tensorflow --> Keras). Ersetze dann die Dateien im /model Ordner mit deinem heruntergeladenen Modell. <br><br>
Da teachlabmachine keine neuen Tensorflow-Versionen unterstützt, müssen wir zuerst eine neue virtuelle Umgebung für dieses Notebook erstellen
1. Öffnen eine Eingabeaufforderung oder ein Terminal
2. Erstelle ein conda environment mit dem Befehl: `conda create -n teachablemachine python=3.10 -y`
3. Aktiviere die Umgebung mit: `conda activate teachablemachine`
4. Installiere die Abhängigkeiten: `pip install "tensorflow==2.12.1" opencv-python`
5. Installiere die Abhängigkeiten: `conda install jupyter`
6. Wähle den richtigen Kernel in der oberen rechten Ecke des Notebooks aus (teachablemachine)

**Schritt 1:** Einbinden von Bibliotheken, um bestimmte Aufgaben ausführen zu können:

In [ ]:
import tensorflow as tf
import cv2
import numpy as np

**Schritt 2:** Lade das Modell und die Label:

In [ ]:
# Pfad zum Ordner, in welchem das Modell liegt
model_dir_relative = "model"

# Wissenschaftliche Notation für eine bessere Übersichtlichkeit deaktivieren
np.set_printoptions(suppress=True)

# Laden des Modells
model = tf.keras.models.load_model(model_dir_relative + "/keras_model.h5", compile=False)

# Laden der Labels
class_names = open(model_dir_relative + "/labels.txt", "r").readlines()

**Schritt 3:** Erstellen des Webcam-Objekts:

In [ ]:
camera_id = 0 # Die id kann 0 oder 1 sein, je nach Standardkamera des Computers
camera = cv2.VideoCapture(camera_id)

**Schritt 4:** Verwenden des Modells für eine Webcam-Bildklassifizierung
* Die vorhandenen Klassen werden in /model/labels.txt gespeichert
* Mit **esc** kann die Bildklassifizierung beendet und die Kamera deaktiviert werden

In [ ]:
while True:
    # Erfassen das Bild der Webkamera.
    ret, image = camera.read()
    image_vis = image.copy()

    # Größe des Bildes in (224-Höhe, 224-Breite) Pixel ändern.
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Umwandels des Bildes in einen Numpy-Array und anpassen des Formats an das Modell.
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalisierung des Bildes
    image = (image / 127.5) - 1

    # Vorhersage des Modells
    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Ausgeben der Vorhersage und des Konfidenzwerts (Wert für Sicherheit der Vorhersage).
    predicted_class = str(class_name[2:]).replace("\n","")
    score = str(np.round(confidence_score * 100))[:-2]
    print("Klasse:", predicted_class, end="")
    print("Konfidenzwert:", score, "%")

    # Anzeigen des Bildes in einem Fenster.
    image_vis = cv2.putText(image_vis, f"Klasse: {predicted_class},   Konfidenzwert: {score}", (30, 30), 0, 1.0, (0, 255, 0), 2)
    cv2.imshow("Webcam Bild mit Klassifizierung", image_vis)


    # Auf einen Tastendruck auf der Tastatur warten.
    keyboard_input = cv2.waitKey(1)

    # Wenn die gedrückte Taste die Taste Esc war, wird die Schleife angehalten.
    # 27 ist der ASCII-Code für die Esc-Taste der Tastatur.
    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()
